In [1]:
# Needed for functions
import numpy as np
import time
from copy import deepcopy

# Import Qiskit classes
import qiskit 
from qiskit import QuantumRegister, QuantumCircuit, ClassicalRegister, Aer
from qiskit.quantum_info import state_fidelity
from qiskit.providers.aer import noise

# Tomography functions
from qiskit.ignis.verification.tomography import state_tomography_circuits, StateTomographyFitter
import qiskit.ignis.mitigation.measurement as mc

In [11]:
#Comparison with my getDensityMatrix Result
q = QuantumRegister(1)
c = ClassicalRegister(1)
circ = QuantumCircuit(q,c)
circ.h(q)
circ.ry(np.pi/3,q)
circ.rx(np.pi,q)

qst_bell = state_tomography_circuits(circ, q)
job = qiskit.execute(qst_bell, Aer.get_backend('qasm_simulator'), shots=5000)

tomo_bell = StateTomographyFitter(job.result(), qst_bell)

tomo_bell.fit()

         ┌───┐┌────────────┐┌────────────┐ ░ ┌─────┐┌───┐┌─┐
q5_0: |0>┤ H ├┤ Ry(1.0472) ├┤ Rx(3.1416) ├─░─┤ Sdg ├┤ H ├┤M├
         └───┘└────────────┘└────────────┘ ░ └─────┘└───┘└╥┘
 c8_0: 0 ═════════════════════════════════════════════════╩═
                                                            
 c7_0: 0 ═══════════════════════════════════════════════════
                                                            


Below is the ideal triplet 0 state, which has a density matrix:
$$\rho_{T} =\begin{bmatrix}   
          0&&0&&0&&0\\
          0&&0.5&&0.5&&0\\
          0&&0.5&&0.5&&0 \\
          0&&0&&0&&0
          \end{bmatrix}$$

In [3]:
# Create the expected density matrix
q2 = QuantumRegister(2)
bell = QuantumCircuit(q2)
bell.x(q2[1])
bell.h(q2[0])
bell.cx(q2[0], q2[1])
print(bell)

job = qiskit.execute(bell, Aer.get_backend('statevector_simulator'))
psi_bell = job.result().get_statevector(bell)
print(psi_bell)

         ┌───┐     
q1_0: |0>┤ H ├──■──
         ├───┤┌─┴─┐
q1_1: |0>┤ X ├┤ X ├
         └───┘└───┘
[0.        +0.j 0.70710678+0.j 0.70710678+0.j 0.        +0.j]


Below, an actual circuit is created where we run state tomography on 2 registers which is a reduced Hilbert space $2^{2}$ instead of $2^{6}$.

In [4]:
# Create the actual circuit 
q2 = QuantumRegister(6)
bell = QuantumCircuit(q2)
bell.x(q2[5])
bell.h(q2[3])
bell.cx(q2[3], q2[5])
print(bell)

                   
q2_0: |0>──────────
                   
q2_1: |0>──────────
                   
q2_2: |0>──────────
         ┌───┐     
q2_3: |0>┤ H ├──■──
         └───┘  │  
q2_4: |0>───────┼──
         ┌───┐┌─┴─┐
q2_5: |0>┤ X ├┤ X ├
         └───┘└───┘


Below the qst_bell circuits are printed out in order to see the kinds of circuits QISKit is using to run the state tomography as opposed to what we used to code the density matrix. QISKit is using 9 circuits to which they apply the gates specified above, and then they run a measurement and apply an $H$ and $S^{t}$ and then do another measurement. The $H$ rotates a qubit 90 degrees about the Y-axis and 180 degrees about the X-axis, while the $S^{t}$ a clockwise $\sqrt{Z}$ rotation. They are using Clifford gates to get the components of the density matrix while in our version of the Density Matrix Code we used the Pauli Matrices to rotate about each axis.

In [5]:
# Generate circuits and run on simulator
t = time.time()
# Generate the state tomography circuits. Only pass in the 
# registers we want to measure (in this case 3 and 5)
qst_bell = state_tomography_circuits(bell, [q2[3],q2[5]])
job = qiskit.execute(qst_bell, Aer.get_backend('qasm_simulator'), shots=5000)
print('Time taken:', time.time() - t)

tomo_bell = StateTomographyFitter(job.result(), qst_bell)
print(qst_bell)
qst_bell[4].draw()

Time taken: 0.2920570373535156
[<qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x000001EF7FC49400>, <qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x000001EF0C28B3C8>, <qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x000001EF0C28B400>, <qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x000001EF0C28B940>, <qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x000001EF0C255898>, <qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x000001EF0C2559B0>, <qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x000001EF0C255400>, <qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x000001EF0C255048>, <qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x000001EF0C226940>]


The fitter outputs a density matrix that corresponds to the number of registers passed in. This makes sense because there were only two registers passed in. The fit fidelity corresponds to the amount of state overlap between the theoretical state and the actual state.

In [8]:
# Perform the tomography fit
# which outputs a density matrix
rho_bell = tomo_bell.fit()
F_bell = state_fidelity(psi_bell, rho_bell)
print(rho_bell)# Density Matrix
print('Fit Fidelity =', F_bell)

np.round_(rho_bell,2)


[[ 2.04952284e-03+0.j          4.43737244e-03-0.00373999j
   3.82841738e-03-0.00426347j  1.67631195e-04+0.00206756j]
 [ 4.43737244e-03+0.00373999j  5.02730560e-01+0.j
   4.97732362e-01-0.00016901j -6.33018959e-03+0.00550803j]
 [ 3.82841738e-03+0.00426347j  4.97732362e-01+0.00016901j
   4.93101837e-01+0.j         -6.87714931e-03+0.00494208j]
 [ 1.67631195e-04-0.00206756j -6.33018959e-03-0.00550803j
  -6.87714931e-03-0.00494208j  2.11807959e-03+0.j        ]]
Fit Fidelity = 0.99564856117508


array([[ 0.  +0.j  ,  0.  -0.j  ,  0.  -0.j  ,  0.  +0.j  ],
       [ 0.  +0.j  ,  0.5 +0.j  ,  0.5 -0.j  , -0.01+0.01j],
       [ 0.  +0.j  ,  0.5 +0.j  ,  0.49+0.j  , -0.01+0.j  ],
       [ 0.  -0.j  , -0.01-0.01j, -0.01-0.j  ,  0.  +0.j  ]])